In [1]:
!pip install pytorch_lightning==0.7.6
!pip install comet-ml

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu Jun 18 17:08:38 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Mount your personal google drive
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
primary_directory = '/content/drive/My Drive/KBP_Challenge'
%cd /content/drive/My\ Drive/KBP_Challenge
%ls

/content/drive/My Drive/KBP_Challenge
checkpoints/  hparams.yaml     options/        src/
data/         lightning_logs/  provided_code/


In [5]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateLogger
from pytorch_lightning.logging import CometLogger
from src.pix2pix_model import Pix2PixModel
from pytorch_lightning.callbacks import ModelCheckpoint
from options.train_options import TrainOptions

In [6]:
checkpoints_dir = '{}/checkpoints'.format(primary_directory)
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoints_dir,
    verbose=True,
    monitor='loss',
    mode='min'
)
lr_logger = LearningRateLogger()

comet_logger = CometLogger(
    api_key="eyAsnp1KA7fXLxFMkEWKjhygS",
    project_name="kbp-challenge",
    workspace="mahruskazi"
)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: Checkpoint directory /content/drive/My Drive/KBP_Challenge/checkpoints exists and is not empty with save_top_k != 0.All files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
CometLogger will be initialized in online mode


In [ ]:
args = ['--batchSize', '8',
        '--which_model_netG', 'unet_128_3d',
        '--which_direction', 'AtoB',
        '--input_nc', '1',
        '--lambda_A', '100',
        '--lr_policy', 'plateau',
        '--epoch_count', '100',
        '--load_epoch', '-1',
        '--lr_decay_iters', '100',
        '--lr', '0.01']

opt = TrainOptions().parse(args)

dataset_dir = '{}/data'.format(primary_directory)
model = Pix2PixModel(opt, dataset_dir, stage='training')

checkpoint = None
if opt.load_epoch != -1:
  checkpoint = '{}/epoch={}.ckpt'.format(checkpoints_dir, opt.load_epoch)

trainer = pl.Trainer(logger=comet_logger,
                     resume_from_checkpoint=checkpoint,
                     gpus=1,
                     checkpoint_callback=checkpoint_callback,
                     callbacks=[lr_logger],
                     max_epochs=opt.epoch_count,
                     check_val_every_n_epoch=3,
                     num_sanity_val_steps=5,
                     val_percent_check=0.5,
                     accumulate_grad_batches=1)
trainer.fit(model)

------------ Options -------------
batchSize: 8
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: False
dataroot: None
dataset_mode: unaligned
display_freq: 200
display_id: 1
display_port: 8097
display_single_pane_ncols: 0
display_winsize: 128
epoch_count: 100
fineSize: 128
gpu_ids: -1
init_type: normal
input_nc: 1
isTrain: True
lambda_A: 100.0
lambda_B: 10.0
lambda_identity: 0.5
loadSize: 128
load_epoch: -1
lr: 0.01
lr_decay_iters: 100
lr_policy: plateau
max_dataset_size: inf
model: pix2pix
nThreads: 2
n_layers_D: 3
name: experiment_name
ndf: 64
ngf: 64
niter: 100
niter_decay: 100
no_dropout: False
no_flip: False
no_html: False
no_img: False
no_lsgan: False
norm: batch_3d
nwf: 64
output_nc: 1
phase: train
pool_size: 50
print_freq: 200
recon_loss: l1
resize_or_crop: resize_and_crop
save_epoch_freq: 5
save_latest_freq: 5000
serial_batches: False
training_size: 100
update_html_freq: 1000
wasserstein: False
which_direction: AtoB
which_epoch: latest
which_model_netD: n_layers_3d
wh

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]


initialization method [normal]


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: You have defined a `test_dataloader()` and have defined a `test_step()`, you may also want to define `test_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/mahruskazi/kbp-challenge/a5473cff4ba64bd39b415ccb23ca2429


   | Name                                                                    | Type                    | Params
----------------------------------------------------------------------------------------------------------------
0  | generator                                                               | UnetGenerator           | 167 M 
1  | generator.model                                                         | UnetSkipConnectionBlock | 167 M 
2  | generator.model.model                                                   | Sequential              | 167 M 
3  | generator.model.model.0        

Number of validation patients: 99


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


100%|██████████| 5/5 [00:03<00:00,  1.38it/s]

{'dvh_score': 21.47698092810773, 'dose_score': 25.13370059466928}
Number of training patients: 100
Number of validation patients: 99



/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
